In [1]:
import numpy as np
import math

File I/O

In [2]:
training_data = np.loadtxt('crime-train.txt', delimiter = '\t', skiprows = 1)
testing_data = np.loadtxt('crime-test.txt', delimiter = '\t', skiprows = 1)

Xtrain = training_data[:,1:]
arr_ones = np.ones(len(Xtrain))
Xtrain = np.column_stack((Xtrain,arr_ones))
Ytrain = training_data[:,0:1]

Xtest = testing_data[:,1:]
arr_ones = np.ones(len(Xtest))
Xtest = np.column_stack((Xtest,arr_ones))
Ytest = testing_data[:,0:1] 

# Ridge Regression with K Fold Cross Validation

In [3]:
def ridge_regression(X,Y,data,Lambda):
    w = np.dot((np.linalg.inv(np.dot(X.T,X) + Lambda*np.identity(len(X.T)))),(np.dot(X.T,Y)))
    prediction_y = []
    for x in data:
        prediction_y.append(np.dot(x.T,w))
    return prediction_y

In [4]:
def RMSE_calc(prediction_y,Y):
    difference = prediction_y - Y
    sum = 0
    for i in difference:
        sum += i**2
    RMSE = math.sqrt(sum/(len(prediction_y)))
    return RMSE

Data for Training Partitions

In [5]:
Xtrain_partition1 = Xtrain[0:319]
Xtrain_partition2 = Xtrain[319:638]
Xtrain_partition3 = Xtrain[638:957]
Xtrain_partition4 = Xtrain[957:1276]
Xtrain_partition5 = Xtrain[1276:1595]

Ytrain_partition1 = Ytrain[0:319]
Ytrain_partition2 = Ytrain[319:638]
Ytrain_partition3 = Ytrain[638:957]
Ytrain_partition4 = Ytrain[957:1276]
Ytrain_partition5 = Ytrain[1276:1595]

Xcomb1 = np.concatenate((Xtrain_partition1,Xtrain_partition2,Xtrain_partition3,Xtrain_partition4))
Xcomb2 = np.concatenate((Xtrain_partition1,Xtrain_partition2,Xtrain_partition3,Xtrain_partition5))
Xcomb3 = np.concatenate((Xtrain_partition1,Xtrain_partition2,Xtrain_partition4,Xtrain_partition5))
Xcomb4 = np.concatenate((Xtrain_partition1,Xtrain_partition3,Xtrain_partition4,Xtrain_partition5))
Xcomb5 = np.concatenate((Xtrain_partition2,Xtrain_partition3,Xtrain_partition4,Xtrain_partition5))

Ycomb1 = np.concatenate((Ytrain_partition1,Ytrain_partition2,Ytrain_partition3,Ytrain_partition4))
Ycomb2 = np.concatenate((Ytrain_partition1,Ytrain_partition2,Ytrain_partition3,Ytrain_partition5))
Ycomb3 = np.concatenate((Ytrain_partition1,Ytrain_partition2,Ytrain_partition4,Ytrain_partition5))
Ycomb4 = np.concatenate((Ytrain_partition1,Ytrain_partition3,Ytrain_partition4,Ytrain_partition5))
Ycomb5 = np.concatenate((Ytrain_partition2,Ytrain_partition3,Ytrain_partition4,Ytrain_partition5))

Calculating Average RMSE values and finding the optimal lambda

In [17]:
avg_RMSE_values = []
min_RMSE = 10
init_lambda = 400
idx = 0
for i in range(10):
    RMSE_values = []
    prediction1 = ridge_regression(Xcomb1,Ycomb1,Xtrain_partition5,init_lambda)
    prediction2 = ridge_regression(Xcomb2,Ycomb2,Xtrain_partition4,init_lambda)
    prediction3 = ridge_regression(Xcomb3,Ycomb3,Xtrain_partition3,init_lambda)
    prediction4 = ridge_regression(Xcomb4,Ycomb4,Xtrain_partition2,init_lambda)
    prediction5 = ridge_regression(Xcomb5,Ycomb5,Xtrain_partition1,init_lambda)
    
    RMSE_values.append(RMSE_calc(prediction1,Ytrain_partition5))
    RMSE_values.append(RMSE_calc(prediction2,Ytrain_partition4))
    RMSE_values.append(RMSE_calc(prediction3,Ytrain_partition3))
    RMSE_values.append(RMSE_calc(prediction4,Ytrain_partition2))
    RMSE_values.append(RMSE_calc(prediction5,Ytrain_partition1))
    
    for j in range(len(RMSE_values)):
        avg_RMSE = (sum(RMSE_values))/5
    avg_RMSE_values.append(avg_RMSE)
    
    init_lambda /= 2

Lambda = 400
for i in avg_RMSE_values:
    print("Lambda", Lambda, ":",i)
    if i < min_RMSE:
        min_RMSE = i
        opt_lambda = Lambda
    Lambda /= 2
idx = avg_RMSE_values.index(min(avg_RMSE_values)) 
print("\nOptimal RMSE:", min_RMSE, "at lambda value =", opt_lambda)


Lambda 400 : 0.14951277619075798
Lambda 200.0 : 0.14069401845317783
Lambda 100.0 : 0.13727677223186574
Lambda 50.0 : 0.13615594392755517
Lambda 25.0 : 0.13591585919851973
Lambda 12.5 : 0.13602237446230672
Lambda 6.25 : 0.13626786494076767
Lambda 3.125 : 0.13657095693642834
Lambda 1.5625 : 0.13688764073197443
Lambda 0.78125 : 0.1371762680686677

Optimal RMSE: 0.13591585919851973 at lambda value = 25.0


The lambda I chose was the lambda value = 25. This is because this lambda gave the smallest average RMSE value.

In [7]:
testing_predictions = ridge_regression(Xtrain,Ytrain,Xtest,opt_lambda)
RMSE_value = RMSE_calc(testing_predictions,Ytest)
print("RMSE_Value for testing data = %s" %RMSE_value)

RMSE_Value for testing data = 0.1457465070705795


Lab 5 Ridge Regression RMSE Using Lambda = 100: 0.14583464490949322

# Linear Regression Using Gradient Descent Algorithm

In [8]:
def gradient_descent(X,Y,w):
    wPlus = w + (alpha*np.dot(X.T,(Y-np.dot(X,w))))
    return wPlus

In [9]:
def lr_loss(X,Y,w):
    loss = np.dot((Y-np.dot(X,w)).T,(Y-np.dot(X,w)))
    return loss

In [10]:
alpha = 0.00001
w0 = np.random.normal(0,1,(Xtrain.shape[1],1))
L1 = 1
L0 = 0
while(abs(L1-L0) > 0.00001):
    wPlus = gradient_descent(Xtrain,Ytrain,w0)
    L0 = lr_loss(Xtrain,Ytrain,w0)
    L1 = lr_loss(Xtrain,Ytrain,wPlus)
    w0 = wPlus

In [11]:
predictions_train = []
predictions_test = []

for x in Xtrain:
    predictions_train.append(np.dot(x.T,w0))
for x in Xtest:
    predictions_test.append(np.dot(x.T,w0))
    
GDtrain_RMSE = RMSE_calc(predictions_train,Ytrain)
GDtest_RMSE = RMSE_calc(predictions_test,Ytest)
print("GD Training RMSE:", GDtrain_RMSE)
print("GD Testing RMSE:", GDtest_RMSE)

GD Training RMSE: 0.12860433042231625
GD Testing RMSE: 0.1474132307679807


Lab 5 Training and Testing RMSE:
    
Training RMSE: 0.1276896742176219

Testing RMSE: 0.14583464490949322

# Ridge Regression Using Gradient Descent and K Fold Cross Validation

Data for Training Partitions

In [12]:
Xtrain_partition1 = Xtrain[0:319]
Xtrain_partition2 = Xtrain[319:638]
Xtrain_partition3 = Xtrain[638:957]
Xtrain_partition4 = Xtrain[957:1276]
Xtrain_partition5 = Xtrain[1276:1595]

Ytrain_partition1 = Ytrain[0:319]
Ytrain_partition2 = Ytrain[319:638]
Ytrain_partition3 = Ytrain[638:957]
Ytrain_partition4 = Ytrain[957:1276]
Ytrain_partition5 = Ytrain[1276:1595]

Xcomb1 = np.concatenate((Xtrain_partition1,Xtrain_partition2,Xtrain_partition3,Xtrain_partition4))
Xcomb2 = np.concatenate((Xtrain_partition1,Xtrain_partition2,Xtrain_partition3,Xtrain_partition5))
Xcomb3 = np.concatenate((Xtrain_partition1,Xtrain_partition2,Xtrain_partition4,Xtrain_partition5))
Xcomb4 = np.concatenate((Xtrain_partition1,Xtrain_partition3,Xtrain_partition4,Xtrain_partition5))
Xcomb5 = np.concatenate((Xtrain_partition2,Xtrain_partition3,Xtrain_partition4,Xtrain_partition5))

Ycomb1 = np.concatenate((Ytrain_partition1,Ytrain_partition2,Ytrain_partition3,Ytrain_partition4))
Ycomb2 = np.concatenate((Ytrain_partition1,Ytrain_partition2,Ytrain_partition3,Ytrain_partition5))
Ycomb3 = np.concatenate((Ytrain_partition1,Ytrain_partition2,Ytrain_partition4,Ytrain_partition5))
Ycomb4 = np.concatenate((Ytrain_partition1,Ytrain_partition3,Ytrain_partition4,Ytrain_partition5))
Ycomb5 = np.concatenate((Ytrain_partition2,Ytrain_partition3,Ytrain_partition4,Ytrain_partition5))

avg_RMSE_values = []
min_RMSE = 10
init_lambda = 400
idx = 0
for i in range(10):
    RMSE_values = []
    prediction1 = ridge_regression(Xcomb1,Ycomb1,Xtrain_partition5,init_lambda)
    prediction2 = ridge_regression(Xcomb2,Ycomb2,Xtrain_partition4,init_lambda)
    prediction3 = ridge_regression(Xcomb3,Ycomb3,Xtrain_partition3,init_lambda)
    prediction4 = ridge_regression(Xcomb4,Ycomb4,Xtrain_partition2,init_lambda)
    prediction5 = ridge_regression(Xcomb5,Ycomb5,Xtrain_partition1,init_lambda)
    
    RMSE_values.append(RMSE_calc(prediction1,Ytrain_partition5))
    RMSE_values.append(RMSE_calc(prediction2,Ytrain_partition4))
    RMSE_values.append(RMSE_calc(prediction3,Ytrain_partition3))
    RMSE_values.append(RMSE_calc(prediction4,Ytrain_partition2))
    RMSE_values.append(RMSE_calc(prediction5,Ytrain_partition1))
    
    for j in range(len(RMSE_values)):
        avg_RMSE = (sum(RMSE_values))/5
    avg_RMSE_values.append(avg_RMSE)
    
    init_lambda /= 2

Lambda = 400
for i in avg_RMSE_values:
    print("Lambda", Lambda, ":",i)
    if i < min_RMSE:
        min_RMSE = i
        opt_lambda = Lambda
    Lambda /= 2
idx = avg_RMSE_values.index(min(avg_RMSE_values)) 
print("\nOptimal RMSE:", min_RMSE, "at lambda value =", opt_lambda)

Lambda 400 : 0.14951277619075798
Lambda 200.0 : 0.14069401845317783
Lambda 100.0 : 0.13727677223186574
Lambda 50.0 : 0.13615594392755517
Lambda 25.0 : 0.13591585919851973
Lambda 12.5 : 0.13602237446230672
Lambda 6.25 : 0.13626786494076767
Lambda 3.125 : 0.13657095693642834
Lambda 1.5625 : 0.13688764073197443
Lambda 0.78125 : 0.1371762680686677

Optimal RMSE: 0.13591585919851973 at lambda value = 25.0


In [13]:
def rr_loss(X,Y,w):
    loss = np.dot((Y-np.dot(X,w)).T,(Y-np.dot(X,w))) + (opt_lambda*np.dot(w.T,w))
    return loss

In [14]:
def gradient_descent_rr(X,Y,w):
    wPlus = w + (alpha*(np.dot(X.T,(Y-np.dot(X,w))) - opt_lambda*w))
    return wPlus

In [15]:
alpha = 0.00001
w0 = np.random.normal(0,1,(Xtrain.shape[1],1))
L1 = 1
L0 = 0
while(abs(L1-L0) > 0.00001):
    wPlus = gradient_descent_rr(Xtrain,Ytrain,w0) 
    L0 = lr_loss(Xtrain,Ytrain,w0)
    L1 = lr_loss(Xtrain,Ytrain,wPlus)
    w0 = wPlus

In [16]:
predictions_train = []
predictions_test = []

for x in Xtrain:
    predictions_train.append(np.dot(x.T,w0))
for x in Xtest:
    predictions_test.append(np.dot(x.T,w0))
    
GDtrain_RMSE = RMSE_calc(predictions_train,Ytrain)
GDtest_RMSE = RMSE_calc(predictions_test,Ytest)
print("Training RMSE:", GDtrain_RMSE)
print("Testing RMSE:", GDtest_RMSE)

Training RMSE: 0.1288712924908713
Testing RMSE: 0.14581250540153343


Lab 5 Training and Testing RMSE:

Training RMSE: 0.13134320424615784

Testing RMSE: 0.14765698468526112

In this lab, we can see that the RMSE values from Lab 6 and Lab 5 are very similar. For linear regression, the lab 5 RMSE values were smaller, which may be because of how the data is formatted. For ridge regression, the lab 6 RMSE values are smaller.